### Prueba con Respuesta para RAG

In [ ]:
from transformers import pipeline
import json

# Cargar el archivo de la jerarquía JSON
with open("grafo_hierarquia.json", "r", encoding="utf-8") as f:
    grafo_json = json.load(f)

# Función de búsqueda
def realizar_consulta(grafo_json, consulta):
    resultados = []
    for cultivo, data in grafo_json.items():
        if consulta.lower() in cultivo.lower():
            resultados.append(f"Encontrado en el cultivo: {cultivo}")
        
        if 'Productos' in data:
            for producto, info_producto in data['Productos'].items():
                if consulta.lower() in producto.lower():
                    resultados.append(f"Encontrado en el producto: {producto} del cultivo {cultivo}")
                
                if 'Controla a' in info_producto:
                    for plaga, detalles in info_producto['Controla a'].items():
                        if consulta.lower() in plaga.lower() or any(consulta.lower() in momento.lower() for momento in detalles['Momento de aplicacion']):
                            resultados.append(f"Producto: {producto} en cultivo {cultivo} controla a la plaga {plaga}")
                            for momento in detalles['Momento de aplicacion']:
                                resultados.append(f"    Momento de aplicacion: {momento}")
    
    return resultados

# Crear contexto de respuesta con resultados de búsqueda
def generar_respuesta(contexto):
    generator = pipeline("text-generation", model="gpt2")
    
    # Limitar el contexto a 512 tokens (aproximadamente) para evitar desbordamientos
    max_context_length = 512
    prompt = "De acuerdo a la búsqueda en nuestra base de datos:\n" + contexto[:max_context_length] + "\n\nLa información relevante es:"

    # Generar la respuesta
    respuesta = generator(prompt, max_new_tokens=100, num_return_sequences=1, truncation=True, pad_token_id=50256)
    return respuesta[0]['generated_text']



De acuerdo a la búsqueda en nuestra base de datos:
Producto: Fastac® en cultivo Maíz controla a la plaga Nezara viridula
    Momento de aplicacion: Aplicar, de acuerdo a los umbrales establecidos por INTA, a partir de 0,4 chinches por metro lineal, cuando la soja esté en estado de formación de vainas (R3). Se recomienda trabajar con altos volúmenes que aseguren una buena penetración en el canopeo, logrando al menos 20 gotas/cm2 en la parte media de la planta.
    Momento de aplicacion: A partir de la floración cuando se observen 2 chinches por metro lineal 

La información relevante es: A partir de de acubar a los muy casa en la parte media. Como es nuevo informión a la planta.

"A PORTION OF BUDGETE IS A NICE MEETUP OF PORTIONS"


In [ ]:
# Ejemplo de uso
consulta = "Soja"
resultados = realizar_consulta(grafo_json, consulta)

# Crear el contexto y generar la respuesta
if resultados:
    contexto = "\n".join(resultados)
    respuesta = generar_respuesta(contexto)
    print(respuesta)
else:
    print("No se encontraron resultados.")


In [19]:
# Genera la respuesta como hasta ahora
respuesta_generada = generar_respuesta(contexto)

# Divide y filtra el texto generado si detecta la frase confusa
respuesta_final = respuesta_generada.split("La información relevante es:")[0]
print(respuesta_final)


De acuerdo a la búsqueda en nuestra base de datos:
Producto: 7344 VT Triple PRO en cultivo Maíz controla a la plaga Spodoptera frugiperda
    Momento de aplicacion: Aplicar cuando se registren 3-4 Isocas por m².
    Momento de aplicacion: Antes de floración, aplicar al observar 15 Isocas por metro lineal de surco y más de 25 % de daño al follaje. Desde floración, aplicar al observar 10-15 Orugas por metro lineal de surco y más de 10 % de daño al follaje.
    Momento de aplicacion: Aplicar en mezcla con herbicidas de presiembra o preemergencia, con incorpora




In [24]:
# Supongamos que `resultados` es una lista de textos con la información obtenida de la base de datos
producto_buscado = "pirate"

# Filtrar resultados que contienen el nombre exacto del producto
resultados_filtrados = [resultado for resultado in resultados if producto_buscado.lower() in resultado.lower()]

if resultados_filtrados:
    contexto = "\n".join(resultados_filtrados)
    respuesta = generar_respuesta(contexto)
    print(respuesta)
else:
    print(f"No se encontró información específica para el producto '{producto_buscado}'.")


De acuerdo a la búsqueda en nuestra base de datos:
Producto: Pirate® en cultivo Maíz controla a la plaga Spodoptera frugiperda
Producto: Pirate® en cultivo Soja controla a la plaga Spodoptera frugiperda

La información relevante es:

Vivello difíncia más más de lo esse.

¿Qué no acuerdo?

¿Querí para específico porque más piedra?


(E+W+S+N.I.)

All Rights Reserved. For non-commercial uses. For additional legal and factual information visit the Anti-Piracy Centre or visit the WorldWideWeb.org, www.


In [25]:
def limpiar_respuesta(texto):
    # Frases que suelen indicar el inicio de contenido irrelevante
    frases_indeseadas = [
        "La información relevante es:", "Mendo", "Mi nuevo", "El de los miños",
        "Lada e los", "ouestrategas", "Dianos nuevas"
    ]
    
    # Cortar el texto en el primer punto donde aparezca alguna de estas frases
    for frase in frases_indeseadas:
        if frase in texto:
            texto = texto.split(frase)[0]
            break  # Deja de buscar más frases después de cortar el texto
    
    # Retorna el texto limpio, sin espacios extra
    return texto.strip()

# Ejemplo de aplicación de limpieza en la respuesta generada
respuesta_generada = """
De acuerdo a la búsqueda en nuestra base de datos:
Momento de aplicacion: Aplicar cuando se observe un 15 % de plantas atacadas (“raspado” de vainas foliares) con larvas en los primeros estadíos (L1 y L2). Puede ser necesaria más de una aplicación en caso de reinfestación.
La información relevante es:
Mendo la entre nueva a los mensos o sus enviados, con la encomer de el bajo, por tu entre o más:
Mi nuevo a los unmas por sus encomer:
El de los miños a la encomer:
Dianos nuevas que se se podítico a los
Lada e los ouestrategas:
"""

# Limpiar la respuesta
respuesta_limpia = limpiar_respuesta(respuesta_generada)
print(respuesta_limpia)


De acuerdo a la búsqueda en nuestra base de datos:
Momento de aplicacion: Aplicar cuando se observe un 15 % de plantas atacadas (“raspado” de vainas foliares) con larvas en los primeros estadíos (L1 y L2). Puede ser necesaria más de una aplicación en caso de reinfestación.


In [31]:
import json

# Cargar el archivo JSON
with open("grafo_hierarquia.json", "r", encoding="utf-8") as f:
    datos_json = json.load(f)

# Función para obtener productos específicos para un cultivo
def obtener_productos_por_cultivo(datos, cultivo):
    productos = []
    if cultivo in datos:
        for producto in datos[cultivo].get("Productos", {}):
            productos.append(producto)
    return productos

# Consultar los productos para soja
cultivo = "Soja"
productos_soja = obtener_productos_por_cultivo(datos_json, cultivo)

# Generar respuesta en lenguaje natural
if productos_soja:
    respuesta = f"Los productos disponibles para el cultivo de {cultivo} son: " + ", ".join(productos_soja)
else:
    respuesta = f"No se encontraron productos específicos para el cultivo de {cultivo}."

print(respuesta)


Los productos disponibles para el cultivo de Soja son: Fastac®, Pirate®, Nomolt®, Acronis®, CZ  3621 STS, CZ 3.45 S STS, CZ 3.45 S STS I, CZ 3.85 S, CZ 3.85 S I, CZ 4.97, CZ 4.97 S I, CZ 4021 STS, CZ 4306, CZ 4306 B I, CZ 4505 B STS I, CZ 4505 STS, CZ 4721 STS, CZ 5407 IPRO, CZ 5407 IPRO I, CZ 5907 IPRO, CZ 5907 IPRO I, CZ 6205, CZ 6205 B I, CZ 6505, CZ 6505 B I, CZ 6806 IPRO, CZ 6806 IPRO I, CZ 7521 IPRO STS, CZ 7905 IPRO, CZ 7905 IPRO I, Duett® Plus, Opera® , Orquesta® Ultra, Priaxor®, Color Coat™, FloRite® 1127, FloRite® 1197, Gestus®, Hicoat® Super, HiStick® Plus, HiStick® Plus + Acronis®, Vault® HP, Vault® HP + Acronis®, Alika®, Imunit®, Fastac® Duo, Atectra® BV, Heat® , Optill®, Zidua® Pack, Interfield®, Pivot® H, Poast®, Basagran 60, Basagran® 60, Dash® MSO MAX, LactoSilo® Gold


In [33]:
import json

# Cargar el archivo JSON
with open("grafo_hierarquia.json", "r", encoding="utf-8") as f:
    datos_json = json.load(f)

# Función para obtener las plagas controladas por un producto específico
def obtener_plagas_por_producto(datos, producto):
    plagas_controladas = {}
    for cultivo, info_cultivo in datos.items():
        for prod, info_producto in info_cultivo.get("Productos", {}).items():
            if producto.lower() == prod.lower():  # Comparar con insensibilidad a mayúsculas
                plagas_controladas[producto] = info_producto.get("Controla a", {})
    return plagas_controladas

# Consultar las plagas controladas por Fastac
producto = "Fastac®"
plagas_controladas = obtener_plagas_por_producto(datos_json, producto)

# Generar respuesta en lenguaje natural
if plagas_controladas:
    respuesta = f"El producto {producto} controla las siguientes plagas:\n"
    for plaga, detalles in plagas_controladas[producto].items():
        respuesta += f"- {plaga}:\n"
    
else:
    respuesta = f"No se encontraron plagas controladas por el producto {producto}."

print(respuesta)


El producto Fastac® controla las siguientes plagas:
- Helicoverpa gelotopoeon:
- Spodoptera frugiperda:
- Anticarsia gemmatalis:
- Nezara viridula:
- Rachiplusia nu:
- Porosagrotis gypaetina:
- Pseudaletia adultera:
- Alabama argillacea:



In [37]:
import json

# Cargar el archivo JSON
with open("grafo_hierarquia.json", "r", encoding="utf-8") as f:
    datos_json = json.load(f)

# Función para obtener las plagas controladas por un producto específico
def obtener_plagas_por_producto(datos, producto):
    plagas_controladas = {}
    for cultivo, info_cultivo in datos.items():
        for prod, info_producto in info_cultivo.get("Productos", {}).items():
            if producto.lower() == prod.lower():  # Comparar con insensibilidad a mayúsculas
                plagas_controladas[producto] = info_producto.get("Controla a", {})
    return plagas_controladas

# Consultar las plagas controladas por Fastac
producto = "Fastac®"
plagas_controladas = obtener_plagas_por_producto(datos_json, producto)

# Generar respuesta en lenguaje natural
if plagas_controladas:
    respuesta = f"El producto {producto} controla las siguientes plagas:\n"
    for plaga, detalles in plagas_controladas[producto].items():
        respuesta += f"- {plaga}:\n"
        for momento in detalles.get("Momento de aplicacion", []):
            respuesta += f"  * Momento de aplicación: {momento}\n"
else:
    respuesta = f"No se encontraron plagas controladas por el producto {producto}."

print(respuesta)


El producto Fastac® controla las siguientes plagas:
- Helicoverpa gelotopoeon:
  * Momento de aplicación: Aplicar cuando se registren 3-4 Isocas por m².
  * Momento de aplicación: Iniciar los tratamientos cuando se detecten 3 o 4 Isocas por metro lineal de surco. Realizar tratamientos terrestres antes de que el cultivo supere los 50 cm de altura, con caudales de 100 Litros/ha.
  * Momento de aplicación: *Adicionar coadyuvante (DASH MSO MAX) a razón de 250 cc/ha. Aplicar cuando se alcance el umbral de 1 a 2 larvas (L1-L2) /m lineal de surco. Estadío del cultivo en estadío vegetativos tempranos. 
- Spodoptera frugiperda:
  * Momento de aplicación: Aplicar cuando se registren 3-4 Isocas por m².
  * Momento de aplicación: Antes de floración, aplicar al observar 15 Isocas por metro lineal de surco y más de 25 % de daño al follaje. Desde floración, aplicar al observar 10-15 Orugas por metro lineal de surco y más de 10 % de daño al follaje.
  * Momento de aplicación: Aplicar en mezcla con her

In [36]:
import json

# Cargar el archivo JSON
with open("grafo_hierarquia.json", "r", encoding="utf-8") as f:
    datos_json = json.load(f)

# Función para obtener el momento de aplicación para un producto, cultivo y plaga específicos
def obtener_momento_aplicacion(datos, cultivo, producto, plaga):
    momentos_aplicacion = []
    
    # Buscar el cultivo y producto especificado
    if cultivo in datos:
        productos = datos[cultivo].get("Productos", {})
        
        # Buscar el producto especificado dentro del cultivo
        if producto in productos:
            plagas = productos[producto].get("Controla a", {})
            
            # Buscar la plaga especificada dentro del producto
            if plaga in plagas:
                momentos_aplicacion = plagas[plaga].get("Momento de aplicacion", [])
    
    return momentos_aplicacion

# Parámetros de la consulta
cultivo = "Maíz"
producto = "Fastac®"
plaga = "Spodoptera frugiperda"

# Obtener el momento de aplicación para el producto, cultivo y plaga especificados
momentos_aplicacion = obtener_momento_aplicacion(datos_json, cultivo, producto, plaga)

# Generar una respuesta en lenguaje natural
if momentos_aplicacion:
    respuesta = f"El momento de aplicación para {producto} en el cultivo {cultivo} contra la plaga {plaga} es:\n"
    for momento in momentos_aplicacion:
        respuesta += f"- {momento}\n"
else:
    respuesta = f"No se encontraron momentos de aplicación para {producto} en el cultivo {cultivo} contra la plaga {plaga}."

print(respuesta)



El momento de aplicación para Fastac® en el cultivo Maíz contra la plaga Spodoptera frugiperda es:
- Aplicar cuando se registren 3-4 Isocas por m².
- Antes de floración, aplicar al observar 15 Isocas por metro lineal de surco y más de 25 % de daño al follaje. Desde floración, aplicar al observar 10-15 Orugas por metro lineal de surco y más de 10 % de daño al follaje.
- Aplicar en mezcla con herbicidas de presiembra o preemergencia, con incorporación superficial. O posteriormente, cuando se observe la presencia de la plaga o daños; en este último caso, aplicar al atardecer, dirigiendo la aplicación a la línea de siembra.
- Aplicar cuando se observe un 15 % de plantas atacadas (“raspado” de vainas foliares) con larvas en los primeros estadíos (L1 y L2). Puede ser necesaria más de una aplicación en caso de reinfestación. Muestreo del cultivo: realizar el monitoreo del lote a partir de 2 hojas en adelante. Observar de 5 a 10 puntos del lote. Evaluar 100 plantas por punto. Se aconseja el ag